<img src="Quantum_Brilliance_dark_blue_Logo_RGB.png" alt="Copyright (c) 2021 Quantum Brilliance Pty Ltd" width="240">

# qbOS Build 0.0721.002
> *Quantum Brilliance Operating System supporting a complete set of quantum computational tasks*

## Superdense Coding Protocol
The superdense coding protocol allows one to **transmit classical information** from one party to another, generally, two classical bits, using a **single qubit**. In a sense, it is a protocol opposite to quantum teleportation.


In [1]:
import xacc
import sys
import qbos

Let Alice and Bob be the two parties that wish to exchange two **classical bits** of information (say '01'):

| Role | Name |
|---|---|
| Receiver | Bob |
| | ^ |
| Message (classical)| '01' |
| | ^ |
| Sender | Alice |

A third party Charlie takes two qubits (`q1` and `q0`) to produce an entangled state by applying a Hadamard gate to his `q0` qubit, which serves as the control qubit for the following CNOT gate on the the target qubit, `q1`.  Next he gives Alice and Bob each a qubit. 


|  | q0 | q1 |
| --- | --- | -- |
| // Performed by Charlie | | |
|| H(q[0]); | |
|| CNOT(q[0],q[1]); | |
| // q0 given to Alice | | |
| // q1 given to Bob | | |



Depending on the pair of classical bits that Alice wishes to send to Bob, she does one of the following operations on her qubit (`q0`):

| Required classical message [c0c1] | Corresponding quantum gates: | q0 |
| --- | --- | --- | 
| '00' | | I(q[0]); |
| '10' | | X(q[0]); |
| '01' | | Z(q[0]); |
| '11' | | X(q[0]);Z(q[0]); |
| | |  |


Alice gives `q0` to Bob, and then Bob proceeds to disentangle and decode the classical message.

### Generate the quantum circuit for the case that '01' is the classical message

In [2]:
generator01 = '''
.compiler xasm
.circuit superdense_coding01
.parameters x
.qbit q
// Charlie generates entangled pair
H(q[0]);
CNOT(q[0], q[1]);

// Alice sends "01"
Z(q[0]);

// Bob disentangles
CNOT(q[0],q[1]);

// Bob decodes
H(q[0]);

// Measurement
Measure(q[0]);
Measure(q[1]);
'''

In [3]:
xacc.qasm(generator01)
superdense01 = xacc.getCompiled('superdense_coding01')
sdstaq = xacc.getCompiler('staq')
import re
q2oqm = re.sub(r"qreg q\[\d+\];", "", sdstaq.translate(superdense01))
qbstr01 = '__qpu__ void QBCIRCUIT(qreg q) {\n' + q2oqm + '}'
print(qbstr01)

__qpu__ void QBCIRCUIT(qreg q) {
OPENQASM 2.0;
include "qelib1.inc";

creg q_c[2];
h q[0];
CX q[0], q[1];
z q[0];
CX q[0], q[1];
h q[0];
measure q[0] -> q_c[0];
measure q[1] -> q_c[1];
}


### Execute the quantum circuit

In [4]:
tqb=qbos.core()
tqb.qb12()
tqb.instring = qbstr01
tqb.run()

### View the decoded message

In [5]:
bit_index = tqb.out_raw[0][0].index(":") - 2
print("Bob receives ",tqb.out_raw[0][0][bit_index],tqb.out_raw[0][0][bit_index-1],sep="")

Bob receives 01


### Generate the quantum circuit for the case that '00' is the classical message

In [6]:
generator00 = '''
.compiler xasm
.circuit superdense_coding00
.parameters x
.qbit q
// Charlie generates entangled pair
H(q[0]);
CNOT(q[0], q[1]);

// Alice sends "00"
// No operation here!

// Bob disentangles
CNOT(q[0],q[1]);

// Bob decodes
H(q[0]);

// Measurement
Measure(q[0]);
Measure(q[1]);
'''

In [7]:
xacc.qasm(generator00)
superdense00 = xacc.getCompiled('superdense_coding00')
import re
q2oqm = re.sub(r"qreg q\[\d+\];", "", sdstaq.translate(superdense00))
qbstr00 = '__qpu__ void QBCIRCUIT(qreg q) {\n' + q2oqm + '}'
print(qbstr00)

__qpu__ void QBCIRCUIT(qreg q) {
OPENQASM 2.0;
include "qelib1.inc";

creg q_c[2];
h q[0];
CX q[0], q[1];
CX q[0], q[1];
h q[0];
measure q[0] -> q_c[0];
measure q[1] -> q_c[1];
}


### Execute the quantum circuit

In [8]:
tqb=qbos.core()
tqb.qb12()
tqb.instring = qbstr00
tqb.run()

### View the decoded message

In [9]:
bit_index = tqb.out_raw[0][0].index(":") - 2
print("Bob receives ",tqb.out_raw[0][0][bit_index],tqb.out_raw[0][0][bit_index-1],sep="")

Bob receives 00
